In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import cv2
from pandas.plotting import table
import matplotlib.pyplot as plt
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
tf.enable_eager_execution()
from DataLoader import DataGenerator,InputGenerator

/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorf

In [3]:
cte_height = 1080;
cte_width = 1920;

In [4]:
Informacoes = pd.read_csv("Depth_0").drop(columns=["Unnamed: 0"])
CU_size = 64
batch = 20

In [5]:
Informacoes['Frame'] = Informacoes['Frame'].astype('uint16')
Informacoes['Height'] = Informacoes['Height'].astype('uint16')
Informacoes['Width'] = Informacoes['Width'].astype('uint16')
Informacoes['QP'] = Informacoes['QP'].astype('uint8')
Informacoes['Split'] = Informacoes['Split'].astype('uint8')

In [6]:
Informacoes["Height_Final"] = Informacoes["Height"]+CU_size
Informacoes["Width_Final"] = Informacoes["Width"]+CU_size
Informacoes.drop(Informacoes.loc[(Informacoes["Height_Final"]>cte_height)|(Informacoes["Width_Final"]>cte_width)].index,inplace=True)
Informacoes.drop(columns=['Height_Final',"Width_Final"],inplace=True)
Informacoes.loc[Informacoes['Origem'].str.contains("OldTown",na=False),"Origem"] = 1
Informacoes.loc[Informacoes['Origem'].str.contains("Crowd",na=False),"Origem"] = 2
Informacoes.loc[Informacoes['Origem'].str.contains("SunFlower",na=False),"Origem"] = 3
Informacoes.loc[Informacoes['Origem'].str.contains("ParkJoy",na=False),"Origem"] = 4
Informacoes.loc[Informacoes['Origem'].str.contains("Ducks",na=False),"Origem"] = 5
Informacoes.loc[Informacoes['Origem'].str.contains("RushHour",na=False),"Origem"] = 6
Informacoes['Origem'] = Informacoes['Origem'].astype('uint8')
Informacoes.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5760000 entries, 0 to 6119969
Data columns (total 6 columns):
Origem    uint8
Frame     uint16
Height    uint16
Width     uint16
QP        uint8
Split     uint8
dtypes: uint16(3), uint8(3)
memory usage: 93.4 MB


In [7]:
parkjoy = np.fromfile('./Videos/ParkJoy_1920x1080_50_8bit_420.yuv',dtype='uint8')
rushhour= np.fromfile('./Videos/RushHour_1920x1080_25_8bit_420.yuv',dtype='uint8')
oldtown = np.fromfile('./Videos/OldTownCross_1920x1080_50_8bit_420.yuv',dtype='uint8')
ducks = np.fromfile('./Videos/DucksTakeOff_1920x1080_50_8bit_420.yuv',dtype='uint8')
crowdrun = np.fromfile('./Videos/Crowdrun_1920x1080_50_8bit_420.yuv',dtype='uint8')
sunflower = np.fromfile('./Videos/Sunflower_1920x1080_25_8bit_420.yuv',dtype='uint8')

In [8]:
Map = {1:oldtown,2:crowdrun,3:sunflower,4:parkjoy,5:ducks,6:rushhour}

In [9]:
Informacoes.head()

,Origem,Frame,Height,Width,QP,Split
0,2,323,0,0,32,1
1,2,323,0,64,32,1
2,2,323,0,128,32,0
3,2,323,0,192,32,0
4,2,323,0,256,32,0


In [9]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(20, activation='sigmoid'),
      tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
  ])

  model.compile(optimizer='sgd',
                loss='mean_squared_error',
                metrics=['accuracy'])
  return model

model = get_compiled_model()

In [10]:
Informacoes = Informacoes.sample(frac=1)
training_set = Informacoes[:int(len(Informacoes)*0.8)]
validation_set = Informacoes[int(len(Informacoes)*0.8):]

In [11]:
training_generator = DataGenerator(training_set,Map,shuffle=False,dim = CU_size,batch_size=batch)
validation_generator = DataGenerator(validation_set,Map,shuffle=False,dim = CU_size,batch_size=batch)

In [12]:
model.fit_generator(generator=training_generator,
                    validation_data = validation_generator,
                    use_multiprocessing=True,
                    workers=3,epochs=2)

Epoch 1/2
230400/230400 [==============================] - 3683s 16ms/step - loss: 0.2479 - acc: 0.5460 - val_loss: 0.2478 - val_acc: 0.5458
Epoch 2/2
230400/230400 [==============================] - 3440s 15ms/step - loss: 0.2478 - acc: 0.5467 - val_loss: 0.2478 - val_acc: 0.5458


In [10]:
model.save("Model_Depth0.h5")

NameError: name 'model' is not defined

In [11]:
modelo = keras.models.load_model("Model_Depth0.h5")

In [25]:
oi = np.empty((1,64*64+1))
modelo.predict(oi)

IndexError: list index out of range